In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
train=pd.read_csv('Training.csv')
train.sort_values(by=['prognosis'])
#print(train)

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
87,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
86,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
85,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,hepatitis A
2312,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,hepatitis A
2311,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,hepatitis A
2339,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,hepatitis A


Calculating diseases probabilities

In [ ]:
diseases=[]
disprob=[]
prognosis=train.loc[:,"prognosis"]
training_size = 4919
#print(prognosis)
name="xxx"
c=0
for i in prognosis:
  if i!=name:
    if(name!="xxx"):
      disprob.append(c/training_size)
    name=i
    diseases.append(name)
    c=1
  else:
    c=c+1
disprob.append(c/training_size)

num_diseases =len(diseases)
#print(nod)
#for i in range(nod):
#  print(diseases[i],disprob[i])

Calculating symptom probabilities

In [ ]:
symprob=[]
for col in train.columns:
  if(col=="prognosis"):break
  val=train.loc[:,col]
  c=0
  for i in val:
    if i==1:
      c=c+1
  symprob.append(c/training_size)

num_symptoms = len(symprob)
#print(symprob)

Calculating symptom conditional probabilities

In [ ]:
sym_condprob = np.zeros((num_diseases,num_symptoms))
name="xxx"
cd=-1 #disease counter
ce=0  #entry counter

for i in train.iloc:
  ce=ce+1
  if i["prognosis"]!=name:
    if(name!="xxx"):
      sym_condprob[cd]=sym_condprob[cd]/ce
    cd=cd+1
    name=i["prognosis"]
    ce=0
  for index in range(num_symptoms):
    val=i[index]
    if val==1:
      sym_condprob[cd][index]=sym_condprob[cd][index]+1



#print(sym_condprob[1][11])

Disease Prediction Method 1

In [ ]:
test=pd.read_csv('Testing.csv')
correct_predictions=0
num_tests=0

for i in test.iloc:
  num_tests=num_tests+1
  current_sympt=[]
  best_prob=0
  pred_dis="xxx"
  for index in range(num_symptoms):
    val=i[index]
    if val==1:
      current_sympt.append(index);
  for j in range(num_diseases):
    tp=disprob[j] #temporary test probability
    for y in current_sympt:
      if(symprob[y]==0):
        continue
      tp=tp*sym_condprob[j][y]/symprob[y]
      #tp=disprob[j]*sym_condprob[y][j]/symprob[y]
    if tp>best_prob:
      best_prop=tp
      pred_dis=diseases[j]
  print("Predicted disease: %40s | Real disease: %40s" % (pred_dis,i["prognosis"]))
  if(pred_dis==i['prognosis']):
    correct_predictions=correct_predictions+1

acc1=correct_predictions/num_tests
#print("\nPrediction accuracy:",acc)


Predicted disease:                         Fungal infection | Real disease:                         Fungal infection
Predicted disease:                                  Allergy | Real disease:                                  Allergy
Predicted disease:                                     GERD | Real disease:                                     GERD
Predicted disease:                      Chronic cholestasis | Real disease:                      Chronic cholestasis
Predicted disease:                            Drug Reaction | Real disease:                            Drug Reaction
Predicted disease:                      Peptic ulcer diseae | Real disease:                      Peptic ulcer diseae
Predicted disease:                                     AIDS | Real disease:                                     AIDS
Predicted disease:                                Diabetes  | Real disease:                                Diabetes 
Predicted disease:                          Gastroenteritis | Re

Disease Prediction Method 2

In [ ]:
results=[]

num_tests=0
correct_predictions=0
for i in test.iloc:
  num_tests=num_tests+1
  current_sympt=[]
  current_prob=[]
  for index in range(num_symptoms):
    val=i[index]
    if val==1:
      current_sympt.append(index);
  for j in range(num_diseases):
    tp=disprob[j] #temporary test probability
    for y in current_sympt:
      if(symprob[y]==0):
        continue
      tp=tp*sym_condprob[j][y]/symprob[y]
      #tp=disprob[j]*sym_condprob[y][j]/symprob[y]
    current_prob.append(tp)

  current_prob=current_prob/sum(current_prob)
  pred_dis=diseases[np.random.choice(num_diseases,1,p=current_prob)[0]]

  print("Predicted disease: %40s | Real disease: %40s" % (pred_dis,i["prognosis"]))
  #iii=index(i["prognosis"])
  res=current_prob[diseases.index(i["prognosis"])]
  results.append(res)
  if(pred_dis==i['prognosis']):
    correct_predictions=correct_predictions+1

acc2=correct_predictions/num_tests
#print("\nPrediction accuracy:",acc)

Predicted disease:                         Fungal infection | Real disease:                         Fungal infection
Predicted disease:                                  Allergy | Real disease:                                  Allergy
Predicted disease:                                     GERD | Real disease:                                     GERD
Predicted disease:                      Chronic cholestasis | Real disease:                      Chronic cholestasis
Predicted disease:                            Drug Reaction | Real disease:                            Drug Reaction
Predicted disease:                      Peptic ulcer diseae | Real disease:                      Peptic ulcer diseae
Predicted disease:                                     AIDS | Real disease:                                     AIDS
Predicted disease:                                Diabetes  | Real disease:                                Diabetes 
Predicted disease:                          Gastroenteritis | Re

In [ ]:
from sklearn.metrics import brier_score_loss
from sklearn.metrics import log_loss
from matplotlib import pyplot
from numpy import array

squ_diff=[(1-i)*(1-i) for i in results]
mean_square_loss=np.mean([(1-i)*(1-i) for i in results])
# print("Mean square loss:",mean_square_loss)

mean_expected_prob=np.mean(results)

print("Prediction accuracy using method 1:",acc1)
print("Prediction accuracy using method 2:",acc2)
print("Mean square loss:",mean_square_loss)
print("Mean expected disease probability (confidence measure):",mean_expected_prob)

Prediction accuracy using method 1: 0.9512195121951219
Prediction accuracy using method 2: 0.975609756097561
Mean square loss: 0.014533354593913616
Mean expected disease probability (confidence measure): 0.9733764349547425
